In [1]:
from run import *

In [2]:
symbol = 'BTC/USDT'

start_date = '2022-12-09'
end_date = '2024-01-01'

In [3]:
df_freq, df_plot_freq, trades_freq, pnl_freq, pnl_concat, pnl_live_concat = run(symbol, start_date, end_date, frequencies=['4h'])

In [4]:
plot_df(pnl_concat)